In [ ]:

import pandas as pd


#Incarcare si curatare dataset

In [ ]:
df = pd.read_csv('sentiment_reviews.csv')

In [ ]:
df['Sentiment'].value_counts()


,count
Sentiment,
1,5000
2,5000
3,5000
4,5000
5,5000


In [ ]:
df = df[df['Sentiment'] != 3]
df['Sentiment_binary'] = df['Sentiment'].apply(lambda x: 1 if x > 3 else 0)

In [ ]:
print(df['Sentiment_binary'].value_counts())

Sentiment_binary
0    10000
1    10000
Name: count, dtype: int64


In [ ]:
df = df.drop(columns=['Sentiment'])


df = df.rename(columns={'Sentiment_binary': 'Sentiment'})


df['Sentiment'] = df['Sentiment'].map({1: 'Positive', 0: 'Negative'})


print(df.head())

                                              Review Sentiment
0  Fast shipping but this product is very cheaply...  Negative
1  This case takes so long to ship and it's not e...  Negative
2  Good for not droids. Not good for iPhones. You...  Negative
3  The cable was not compatible between my macboo...  Negative
4  The case is nice but did not have a glow light...  Negative


In [ ]:
df['Sentiment'].value_counts()

,count
Sentiment,
Negative,10000
Positive,10000


In [ ]:
df.head()

,Review,Sentiment
0,Fast shipping but this product is very cheaply...,Negative
1,This case takes so long to ship and it's not e...,Negative
2,Good for not droids. Not good for iPhones. You...,Negative
3,The cable was not compatible between my macboo...,Negative
4,The case is nice but did not have a glow light...,Negative


#Impartirea datelor

In [ ]:
X = df['Review']
y = df['Sentiment']

#Preprocesarea textului

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):

    if not isinstance(text, str):
        return ""

    #Lowercase
    text = text.lower()

    #remove punctuation marks
    text = text.translate(str.maketrans('', '', string.punctuation))

    #tokenizare
    tokens = word_tokenize(text)

    #remove stopwords
    tokens = [word for word in tokens if word not in stop_words]

    #lemmanizare
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)


X_processed = X.apply(preprocess_text)

X_processed.head()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,Review
0,fast shipping product cheaply made brought gra...
1,case take long ship even worth dont buy
2,good droids good iphones use feature watch iph...
3,cable compatible macbook iphone also connector...
4,case nice glow light im disappointed product d...


In [ ]:
df["Sentiment"].value_counts()

,count
Sentiment,
Negative,10000
Positive,10000


In [ ]:
df['Clean_Text'] = df['Review'].apply(preprocess_text)


In [ ]:
df.head()

,Review,Sentiment,Clean_Text
0,Fast shipping but this product is very cheaply...,Negative,fast shipping product cheaply made brought gra...
1,This case takes so long to ship and it's not e...,Negative,case take long ship even worth dont buy
2,Good for not droids. Not good for iPhones. You...,Negative,good droids good iphones use feature watch iph...
3,The cable was not compatible between my macboo...,Negative,cable compatible macbook iphone also connector...
4,The case is nice but did not have a glow light...,Negative,case nice glow light im disappointed product d...


In [ ]:
#transformare in array uri de tokeni
from nltk.tokenize import word_tokenize


sentences = df['Clean_Text'].apply(word_tokenize).tolist()


In [ ]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 52.7 MB/s eta 0:00:00


In [ ]:
from gensim.models import Word2Vec

#CBOW
w2v_cbow = Word2Vec(
    sentences,
    vector_size=100,
    window=5,
    min_count=1,
    workers=4,
    sg=0
)

#Skip Gram
w2v_skip = Word2Vec(
    sentences,
    vector_size=100,
    window=5,
    min_count=1,
    workers=4,
    sg=1
)


In [ ]:
import numpy as np
#transformare text in vector numeric
def review_vector(tokens, model):
    vectors = []
    for t in tokens:
        if t in model.wv:
            vectors.append(model.wv[t])
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)


In [ ]:
df['W2V_CBOW_Vector'] = df['Clean_Text'].apply(
    lambda x: review_vector(word_tokenize(x), w2v_cbow)
)
df['W2V_SkipGram_Vector'] = df['Clean_Text'].apply(
    lambda x: review_vector(word_tokenize(x), w2v_skip)
)


In [ ]:
df.head()

,Review,Sentiment,Clean_Text,W2V_CBOW_Vector,W2V_SkipGram_Vector
0,Fast shipping but this product is very cheaply...,Negative,fast shipping product cheaply made brought gra...,"[-0.40108344, 0.59246874, 0.21978195, 0.362995...","[-0.15478472, 0.26678887, 0.15037446, 0.282382..."
1,This case takes so long to ship and it's not e...,Negative,case take long ship even worth dont buy,"[-0.9109252, 0.9431109, 0.6043023, 0.48559725,...","[-0.32103208, 0.1869572, 0.271396, 0.37035164,..."
2,Good for not droids. Not good for iPhones. You...,Negative,good droids good iphones use feature watch iph...,"[-0.1620059, 1.0460296, -0.13275398, 0.2516529...","[-0.01990341, 0.2739495, 0.08467096, 0.2279769..."
3,The cable was not compatible between my macboo...,Negative,cable compatible macbook iphone also connector...,"[-0.20537704, 0.9854769, 0.06994826, 0.5307793...","[-0.13851188, 0.3956825, 0.033697113, 0.252212..."
4,The case is nice but did not have a glow light...,Negative,case nice glow light im disappointed product d...,"[-0.5663855, 0.7447072, 0.33229417, -0.0616118...","[-0.24249744, 0.17199352, 0.12976612, 0.177721..."


In [ ]:
#FastText
from gensim.models import FastText

fasttext_model = FastText(
    sentences,
    vector_size=100,
    window=5,
    min_count=1,
    workers=4,
    sg=1    #
)


In [ ]:
import numpy as np

def review_vector(tokens, model):
    vectors = []
    for t in tokens:
        if t in model.wv:
            vectors.append(model.wv[t])
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)


In [ ]:
df['FastText_Vector'] = df['Clean_Text'].apply(
    lambda x: review_vector(word_tokenize(x), fasttext_model)
)


In [ ]:
df.head()

,Review,Sentiment,Clean_Text,W2V_CBOW_Vector,W2V_SkipGram_Vector,FastText_Vector
0,Fast shipping but this product is very cheaply...,Negative,fast shipping product cheaply made brought gra...,"[-0.40108344, 0.59246874, 0.21978195, 0.362995...","[-0.15478472, 0.26678887, 0.15037446, 0.282382...","[0.1282819, 0.0015799372, -0.27923736, 0.09458..."
1,This case takes so long to ship and it's not e...,Negative,case take long ship even worth dont buy,"[-0.9109252, 0.9431109, 0.6043023, 0.48559725,...","[-0.32103208, 0.1869572, 0.271396, 0.37035164,...","[0.14669672, 0.2088346, -0.24307613, 0.2185993..."
2,Good for not droids. Not good for iPhones. You...,Negative,good droids good iphones use feature watch iph...,"[-0.1620059, 1.0460296, -0.13275398, 0.2516529...","[-0.01990341, 0.2739495, 0.08467096, 0.2279769...","[0.062483825, 0.155946, -0.3404017, 0.08098066..."
3,The cable was not compatible between my macboo...,Negative,cable compatible macbook iphone also connector...,"[-0.20537704, 0.9854769, 0.06994826, 0.5307793...","[-0.13851188, 0.3956825, 0.033697113, 0.252212...","[0.18600665, 0.20512109, -0.30052134, -0.01496..."
4,The case is nice but did not have a glow light...,Negative,case nice glow light im disappointed product d...,"[-0.5663855, 0.7447072, 0.33229417, -0.0616118...","[-0.24249744, 0.17199352, 0.12976612, 0.177721...","[0.15732364, 0.056685895, -0.2392246, -2.01031..."


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

#Feature Matrix
X_cbow = np.vstack(df['W2V_CBOW_Vector'].values)
X_skip = np.vstack(df['W2V_SkipGram_Vector'].values)
X_fast = np.vstack(df['FastText_Vector'].values)


y = df['Sentiment']

#Splitting train si test
X_cbow_train, X_cbow_test, y_train, y_test = train_test_split(X_cbow, y, test_size=0.2, random_state=42)
X_skip_train, X_skip_test, _, _ = train_test_split(X_skip, y, test_size=0.2, random_state=42)
X_fast_train, X_fast_test, _, _ = train_test_split(X_fast, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

gnb = GaussianNB()


#Antrenarea Modelelor

In [ ]:
#Word2Vec CBOW si Naive Bayes
gnb.fit(X_cbow_train, y_train)
y_pred_cbow = gnb.predict(X_cbow_test)

print("GaussianNB – Word2Vec CBOW")
print(classification_report(y_test, y_pred_cbow))


GaussianNB – Word2Vec CBOW
              precision    recall  f1-score   support

    Negative       0.63      0.78      0.70      2019
    Positive       0.71      0.53      0.61      1981

    accuracy                           0.66      4000
   macro avg       0.67      0.66      0.65      4000
weighted avg       0.67      0.66      0.65      4000



In [ ]:
#WordWvex SkipGram si Naive Bayes
gnb.fit(X_skip_train, y_train)
y_pred_skip = gnb.predict(X_skip_test)

print("GaussianNB – Word2Vec Skip-Gram")
print(classification_report(y_test, y_pred_skip))


NameError: name 'gnb' is not defined

In [ ]:
#FastText cu Niave Bayes
gnb.fit(X_fast_train, y_train)
y_pred_fast = gnb.predict(X_fast_test)

print("GaussianNB – FastText")
print(classification_report(y_test, y_pred_fast))


GaussianNB – FastText
              precision    recall  f1-score   support

    Negative       0.66      0.82      0.73      2019
    Positive       0.76      0.58      0.65      1981

    accuracy                           0.70      4000
   macro avg       0.71      0.70      0.69      4000
weighted avg       0.71      0.70      0.69      4000



In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel="linear")


In [ ]:
#Word2vec cu CBOW si SVM
svm.fit(X_cbow_train, y_train)
y_pred_cbow_svm = svm.predict(X_cbow_test)

print("SVM – Word2Vec CBOW")
print(classification_report(y_test, y_pred_cbow_svm))


SVM – Word2Vec CBOW
              precision    recall  f1-score   support

    Negative       0.73      0.81      0.77      2019
    Positive       0.78      0.70      0.74      1981

    accuracy                           0.75      4000
   macro avg       0.76      0.75      0.75      4000
weighted avg       0.76      0.75      0.75      4000



In [ ]:
#Word2Vec SVM cu Skip Gram

svm.fit(X_skip_train, y_train)
y_pred_skip_svm = svm.predict(X_skip_test)

print("SVM – Word2Vec Skip-Gram")
print(classification_report(y_test, y_pred_skip_svm))


NameError: name 'svm' is not defined

In [ ]:
#FastText cu SVM

svm.fit(X_fast_train, y_train)
y_pred_fast_svm = svm.predict(X_fast_test)

print("SVM – FastText")
print(classification_report(y_test, y_pred_fast_svm))


SVM – FastText
              precision    recall  f1-score   support

    Negative       0.79      0.82      0.80      2019
    Positive       0.81      0.78      0.79      1981

    accuracy                           0.80      4000
   macro avg       0.80      0.80      0.80      4000
weighted avg       0.80      0.80      0.80      4000



#TABEL

https://umftgm-my.sharepoint.com/:x:/r/personal/barstan_mihnea-filip_25_stud_umfst_ro/_layouts/15/doc.aspx?sourcedoc=%7Bfc34e1f2-815a-4cfa-b6e3-abc8fb4b1532%7D&action=edit

Analiza comparativa

a) Vectorizarea care a functionat cel mai bine a fost FastText SVM

b) Svm a dat rezultate mai bune decat naive bayes la toate

c) CBOW a fost mai rapid decat skipgram inn timpul antrenarii si din ce am citit produce embedding uri mai stabile

d) fastext descompunu fiecare cuvant in sub-cuvinte poate genera embedding uri pentru un cuvant nevazut combinand embedding urile subcuvintelor sale

e) Da, se poate vedea din rezultatele obtinute

f) Daca embedding urile sunt mai bune vectorii reprezinta mai precis sensul cuvintelor. Modelele pot distinge mai bine intre sentimente.